### Importation des librairies

In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# librairies necessaires pour centrer reduire les variables 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# librairie pour le réequilibrage smote
from imblearn.over_sampling import SMOTENC

# librairies necessaires aux modeles de prédiction & cross validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

### Importation & assemblage des bases de donnees

In [69]:
#data = pd.read_csv('C:/Users/win/Documents/M2 SEP/S1/Cousin/Donnees_equilibrees.csv',sep = ",")
stat = pd.read_csv('../Les_données/operations_stats.csv', sep=";",encoding = 'utf-8')
ope = pd.read_csv('../Les_données/operations.csv',sep = ";",encoding = 'utf-8')
flot = pd.read_csv('../Les_données/flotteurs.csv',sep = ",",)
res_humain = pd.read_csv('../Les_données/resultats_humain.csv',sep = ";",encoding = 'utf-8')

stat.head()

C:\Users\Alizee\AppData\Local\Temp\ipykernel_11040\2964632729.py:2: DtypeWarning: Columns (0,5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  stat = pd.read_csv('../Les_données/operations_stats.csv', sep=";",encoding = 'utf-8')


,operation_id,mois,mois_texte,est_weekend,est_jour_ferie,est_vacances_scolaires,phase_journee,distance_cote_metres,distance_cote_milles_nautiques,maree_port,maree_coefficient,maree_categorie,nombre_personnes_assistees,nombre_personnes_tous_deces,nombre_personnes_tous_deces_ou_disparues,mort,nombre_personnes_impliquees
0,-134138,9,Septembre,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,2
1,-134137,9,Septembre,False,False,False,nuit,0.0,0.00,NaN,NaN,NaN,0,1,1,1,2
2,-134136,9,Septembre,False,False,False,dejeuner,11500.0,6.21,NaN,NaN,NaN,8,0,0,0,9
3,-134135,9,Septembre,False,False,False,matinee,51700.0,27.92,NaN,NaN,NaN,0,0,0,0,2
4,-134134,9,Septembre,False,False,False,nuit,9600.0,5.18,NaN,NaN,NaN,0,0,0,0,1


In [70]:
# assemblage des 4 bases de données
join1 = pd.merge(stat,ope,on = 'operation_id',how='inner')
join2 = pd.merge(flot,join1,on = 'operation_id',how='inner')
data = pd.merge(res_humain,join2,on = 'operation_id',how='inner')

data.head()

,operation_id,categorie_personne,type_flotteur,categorie_flotteur,mois,mois_texte,est_weekend,est_jour_ferie,est_vacances_scolaires,phase_journee,...,maree_categorie,nombre_personnes_assistees,nombre_personnes_tous_deces,nombre_personnes_tous_deces_ou_disparues,mort,nombre_personnes_impliquees,departement,zone_responsabilite,vent_force,mer_force
0,-134138,Plaisancier francais,Plaisance aÂ voile,Plaisance,9,Septembre,False,False,False,NaN,...,NaN,0,0,0,0,2,NaN,Responsabilite francaise,NaN,NaN
1,-134137,Autre,Canoe / Kayak / Aviron,Loisir nautique,9,Septembre,False,False,False,nuit,...,NaN,0,1,1,1,2,Mayotte,Plage et 300 metres,1.0,3.0
2,-134137,Autre,Canoe / Kayak / Aviron,Loisir nautique,9,Septembre,False,False,False,nuit,...,NaN,0,1,1,1,2,Mayotte,Plage et 300 metres,1.0,3.0
3,-134136,Clandestin,Plaisance aÂ moteur < 8m,Plaisance,9,Septembre,False,False,False,dejeuner,...,NaN,8,0,0,0,9,Mayotte,Eaux territoriales,2.0,4.0
4,-134136,Clandestin,Plaisance aÂ moteur < 8m,Plaisance,9,Septembre,False,False,False,dejeuner,...,NaN,8,0,0,0,9,Mayotte,Eaux territoriales,2.0,4.0


In [71]:
test = data.copy()

In [72]:
test = test.dropna(axis=0) # suppression des lignes ayant des valeurs manquantes
test.shape # connaitre les dimensions de notre dataframe 

(55359, 24)

In [73]:
#Valeur Target
data['mort'].value_counts() # compter le nombre de données appartenant à chaque classe

0    193592
1      5826
Name: mort, dtype: int64

### Préparation des données

In [74]:
#Convertir la variable "est_vacances_scolaires" en booléen
test["est_vacances_scolaires"] = test["est_vacances_scolaires"].astype("bool")
test.dtypes

operation_id                                 object
categorie_personne                           object
type_flotteur                                object
categorie_flotteur                           object
mois                                          int64
mois_texte                                   object
est_weekend                                    bool
est_jour_ferie                                 bool
est_vacances_scolaires                         bool
phase_journee                                object
distance_cote_metres                        float64
distance_cote_milles_nautiques              float64
maree_port                                   object
maree_coefficient                           float64
maree_categorie                              object
nombre_personnes_assistees                    int64
nombre_personnes_tous_deces                   int64
nombre_personnes_tous_deces_ou_disparues      int64
mort                                          int64
nombre_perso

In [75]:
#On selectionne les flottants
flottant = list(test.select_dtypes('float').columns)
flottant

['distance_cote_metres',
 'distance_cote_milles_nautiques',
 'maree_coefficient',
 'vent_force',
 'mer_force']

In [76]:
#On selectionne les objets
objet = list(test.select_dtypes('object').columns)
objet

['operation_id',
 'categorie_personne',
 'type_flotteur',
 'categorie_flotteur',
 'mois_texte',
 'phase_journee',
 'maree_port',
 'maree_categorie',
 'departement',
 'zone_responsabilite']

In [77]:
#On selectionne les booleens
booleen =list(test.select_dtypes('bool').columns)
booleen

['est_weekend', 'est_jour_ferie', 'est_vacances_scolaires']

In [78]:
#On selectionne les valeurs quali qu'on veut
key_columns= ['mois','mort','departement','categorie_personne',
 'categorie_flotteur',
 'phase_journee',
 'maree_port']

In [79]:
#On créé df composé des booleens et des quali
df_quali = test[key_columns + booleen ]
df_quali

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires
2284,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True
2326,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True
2347,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True
2356,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
2357,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
...,...,...,...,...,...,...,...,...,...,...
190201,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True
190202,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True
190203,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True
190204,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False


In [80]:
#On reset les index 
df_quali = df_quali.reset_index(drop=True)
df_quali

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires
0,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True
1,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True
2,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True
3,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
4,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
...,...,...,...,...,...,...,...,...,...,...
55354,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True
55355,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True
55356,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True
55357,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False


### Centrer et réduire les variables quantitatives

In [81]:
scaler = StandardScaler() # fonction qui centre et réduit les variables

convert = test[flottant] # selection des variables quanti à centrer-réduire, obligatoire pour le SMOTE

In [82]:
df_scaled = scaler.fit_transform(convert.to_numpy())
df_scaled

array([[-0.54311675, -0.54306593,  1.08764217, -0.34275064, -0.70052844],
       [ 3.98660057,  3.98649854, -0.69133547,  1.60293565,  1.42647301],
       [-0.54311675, -0.54306593, -0.93841569,  0.30581146,  0.36297229],
       ...,
       [ 0.56989951,  0.56895413, -0.59250338, -0.34275064, -1.76402917],
       [-0.31015985, -0.30819963, -0.93841569, -1.63987483,  0.36297229],
       [ 0.7252041 ,  0.7271294 ,  0.34640149, -1.63987483, -1.76402917]])

In [83]:
df_scaled = pd.DataFrame(df_scaled, columns = flottant) # convertir en Data Frame
df_scaled

,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,-0.543117,-0.543066,1.087642,-0.342751,-0.700528
1,3.986601,3.986499,-0.691335,1.602936,1.426473
2,-0.543117,-0.543066,-0.938416,0.305811,0.362972
3,3.650107,3.650975,-0.493671,-0.342751,0.362972
4,3.650107,3.650975,-0.493671,-0.342751,0.362972
...,...,...,...,...,...
55354,-0.517233,-0.519100,0.000489,0.305811,0.362972
55355,-0.154855,-0.154818,0.000489,-0.342751,0.362972
55356,0.569900,0.568954,-0.592503,-0.342751,-1.764029
55357,-0.310160,-0.308200,-0.938416,-1.639875,0.362972


In [84]:
test = pd.DataFrame(scaler.inverse_transform(df_scaled),columns=flottant)
test

,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,0.0,2.220446e-16,92.0,3.0,2.0
1,17500.0,9.450000e+00,56.0,6.0,4.0
2,0.0,2.220446e-16,51.0,4.0,3.0
3,16200.0,8.750000e+00,60.0,3.0,3.0
4,16200.0,8.750000e+00,60.0,3.0,3.0
...,...,...,...,...,...
55354,100.0,5.000000e-02,70.0,4.0,3.0
55355,1500.0,8.100000e-01,70.0,3.0,3.0
55356,4300.0,2.320000e+00,58.0,3.0,1.0
55357,900.0,4.900000e-01,51.0,1.0,3.0


In [85]:
df2 = pd.concat([df_quali,df_scaled],axis = 1,) # création de df2, notre dataframe centré réduit
df2

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True,-0.543117,-0.543066,1.087642,-0.342751,-0.700528
1,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True,3.986601,3.986499,-0.691335,1.602936,1.426473
2,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True,-0.543117,-0.543066,-0.938416,0.305811,0.362972
3,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
4,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55354,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True,-0.517233,-0.519100,0.000489,0.305811,0.362972
55355,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True,-0.154855,-0.154818,0.000489,-0.342751,0.362972
55356,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True,0.569900,0.568954,-0.592503,-0.342751,-1.764029
55357,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False,-0.310160,-0.308200,-0.938416,-1.639875,0.362972


In [86]:
df2['mort'].value_counts(normalize = True) # On vérifie si la variable Mort est toujours ok

0    0.977203
1    0.022797
Name: mort, dtype: float64

In [87]:
df =df2 #On met df2 dans df
df

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True,-0.543117,-0.543066,1.087642,-0.342751,-0.700528
1,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True,3.986601,3.986499,-0.691335,1.602936,1.426473
2,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True,-0.543117,-0.543066,-0.938416,0.305811,0.362972
3,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
4,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55354,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True,-0.517233,-0.519100,0.000489,0.305811,0.362972
55355,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True,-0.154855,-0.154818,0.000489,-0.342751,0.362972
55356,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True,0.569900,0.568954,-0.592503,-0.342751,-1.764029
55357,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False,-0.310160,-0.308200,-0.938416,-1.639875,0.362972


### Réequilibrage SMOTE 

In [51]:
smo = SMOTENC(categorical_features=[1,2,3,4,5], sampling_strategy=1, k_neighbors=2)
X = df.drop('mort', axis=1)
y = df['mort']

X_res, y_res = smo.fit_resample(X, y)

In [52]:
df = pd.concat([X_res,y_res],axis=1)
df
#On concatene les 2

df=df.sample(frac=1).reset_index(drop=True)
df

#Je remélange mes données,

,mois,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force,mort
0,7,Loire-Atlantique,Toutes categories,Loisir nautique,apres-midi,Saint-Nazaire,True,False,True,-0.543117,-0.543066,-1.234912,0.305811,-0.700528,1
1,3,Pas-de-Calais,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True,-0.543117,-0.543066,1.038226,-0.907525,-0.700528,1
2,3,Morbihan,Toutes categories,Loisir nautique,apres-midi,Port-Navalo,True,False,True,-0.498813,-0.497943,2.173412,-0.991313,-0.700528,1
3,5,Cotes-d'Armor,Toutes categories,Plaisance,apres-midi,Brest,False,False,False,3.442355,3.441355,-0.173116,-0.400163,-0.700528,1
4,8,Morbihan,Toutes categories,Plaisance,matinee,Pointe de Grave,False,True,True,-0.543117,-0.543066,0.815483,0.709947,1.426473,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108189,7,Pas-de-Calais,Autre,Loisir nautique,apres-midi,Wissant,True,False,True,-0.387812,-0.389684,-0.048927,-2.288437,-2.827530,0
108190,6,Somme,Autre,Plaisance,apres-midi,Le Touquet,False,False,False,3.675991,3.674941,-1.037248,-0.342751,0.362972,1
108191,6,Morbihan,Toutes categories,Loisir nautique,apres-midi,Port-Navalo,False,False,False,-0.294287,-0.293306,0.323219,0.305811,0.362972,1
108192,3,Gironde,Toutes categories,Peche,matinee,Arcachon Eyrac,True,False,True,-0.543117,-0.543066,-1.351906,0.067442,0.362972,1


In [53]:
#Test remettre en forme 
essai_key = df[key_columns]
essai_flottant = df[flottant]
essay_bool = df[booleen]

In [54]:
tente = pd.DataFrame(scaler.inverse_transform(essai_flottant, copy=None), columns= flottant)
tente

,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,0.000000,2.220446e-16,45.000000,4.000000e+00,2.0
1,0.000000,2.220446e-16,91.000000,2.129190e+00,2.0
2,171.162018,9.413911e-02,113.972012,2.000000e+00,2.0
3,15397.374641,8.312671e+00,66.486873,2.911478e+00,2.0
4,0.000000,2.220446e-16,86.492502,4.623126e+00,4.0
...,...,...,...,...,...
108189,600.000000,3.200000e-01,69.000000,4.440892e-16,0.0
108190,16300.000000,8.800000e+00,49.000000,3.000000e+00,3.0
108191,961.322376,5.210729e-01,76.530879,4.000000e+00,3.0
108192,0.000000,2.220446e-16,42.632465,3.632465e+00,3.0


In [55]:
ensemble = pd.concat([essai_key,tente,essay_bool],axis=1)
df = ensemble
df

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force,est_weekend,est_jour_ferie,est_vacances_scolaires
0,7,1,Loire-Atlantique,Toutes categories,Loisir nautique,apres-midi,Saint-Nazaire,0.000000,2.220446e-16,45.000000,4.000000e+00,2.0,True,False,True
1,3,1,Pas-de-Calais,Toutes categories,Plaisance,nuit,Arcachon Eyrac,0.000000,2.220446e-16,91.000000,2.129190e+00,2.0,False,False,True
2,3,1,Morbihan,Toutes categories,Loisir nautique,apres-midi,Port-Navalo,171.162018,9.413911e-02,113.972012,2.000000e+00,2.0,True,False,True
3,5,1,Cotes-d'Armor,Toutes categories,Plaisance,apres-midi,Brest,15397.374641,8.312671e+00,66.486873,2.911478e+00,2.0,False,False,False
4,8,1,Morbihan,Toutes categories,Plaisance,matinee,Pointe de Grave,0.000000,2.220446e-16,86.492502,4.623126e+00,4.0,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108189,7,0,Pas-de-Calais,Autre,Loisir nautique,apres-midi,Wissant,600.000000,3.200000e-01,69.000000,4.440892e-16,0.0,True,False,True
108190,6,1,Somme,Autre,Plaisance,apres-midi,Le Touquet,16300.000000,8.800000e+00,49.000000,3.000000e+00,3.0,False,False,False
108191,6,1,Morbihan,Toutes categories,Loisir nautique,apres-midi,Port-Navalo,961.322376,5.210729e-01,76.530879,4.000000e+00,3.0,False,False,False
108192,3,1,Gironde,Toutes categories,Peche,matinee,Arcachon Eyrac,0.000000,2.220446e-16,42.632465,3.632465e+00,3.0,True,False,True


In [56]:
#Valeur Target
df['mort'].value_counts()

1    54097
0    54097
Name: mort, dtype: int64

### Réequilibrage aléatoire sans remise

In [96]:
morts = df2.loc[df2['mort']==1,:] # création bdd contenant que les observations "morts"
vivants = df2.loc[df2['mort']==0,:] # création bdd contenant que les observations "vivants"

# tirer aleatoirement sans remise tel qu'on ai le meme nombre de données dans les bases de données morts et vivants 
vivants_equilibre = vivants.sample(morts.shape[0])
data_equilibre = pd.concat([vivants_equilibre,morts])  
data_equilibre.head()

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
50052,3,0,Morbihan,Toutes categories,Loisir nautique,apres-midi,Port-Navalo,True,False,True,-0.543117,-0.543066,-0.098343,1.602936,1.426473
6678,7,0,Nord,Plaisancier francais,Plaisance,apres-midi,Dunkerque,False,False,False,-0.543117,-0.543066,1.087642,-0.342751,0.362972
1563,8,0,Loire-Atlantique,Autre,Loisir nautique,apres-midi,Saint-Nazaire,True,False,True,-0.439580,-0.437616,1.532387,0.305811,-0.700528
17616,11,0,Charente-Maritime,Plaisancier francais,Plaisance,dejeuner,La Rochelle-Pallice,False,False,False,-0.543117,-0.543066,-1.531408,-0.991313,0.362972
31792,10,0,Manche,Toutes categories,Peche,nuit,Ouistreham,True,False,True,-0.543117,-0.543066,-0.345423,-0.342751,-1.764029


In [98]:
# mélanger les observations dans la bdd equilibrée. 
data_equilibre = data_equilibre.sample(frac=1) # frac=1 signifie retourner toutes les lignes (dans un ordre aléatoire).

### Fonctions qui recodent, supprime les NA et créent les bases X et Y 

In [58]:
# encodage des variables qualitatives

def encodage(df):
   code = {"Alpes-Maritimes":0,
"Aude":1,
"Bouches-du-Rhone":2,
"Calvados":3,
"Charente-Maritime":4,
"Collectivite de Saint Barthelemy":5,
"Collectivite de Saint Martin":6,
"Corse":7,
"Corse-du-Sud":8,
"Cotes-d'Armor":9,
"Eure":10,
"Finistere":11,
"Gard":12,
"Gironde":13,
"Guadeloupe":14,
"Guyane":15,
"Haute-Corse":16,
"Herault":17,
"Ille-et-Vilaine":18,
"Landes":19,
"Loire-Atlantique":20,
"Manche":21,
"Martinique":22,
"Mayotte":23,
"Morbihan":24,
"Nord":25,
"Nouvelle-Caledonie":26,
"Pas-de-Calais":27,
"Polynesie":28,
"Pyrenees-Atlantiques":29,
"Pyrenees-Orientales":30,
"Reunion":31,
"Saint-Pierre-et-Miquelon":32,
"Seine-Maritime":33,
"Somme":34,
"Var":35,
"Vendee":36,

"Eaux territoriales":0,
"Plage et 300 metres":1,
"Port et acces":2,
"Responsabilite francaise":3,
"Autre":0,
           
"Clandestin":1,
"Commerce francais":2,
"Marin etranger":3,
"Pecheur francais":4,
"Plaisancier francais":5,
"Toutes categories":6,
                 
"Aeronef":1,
"Commerce":2,
"Loisir nautique":3,
"Peche":4,
"Plaisance":5,
           
"apres-midi":0,
"dejeuner":1,
"matinee":2,
"nuit":3,
                
"Arcachon Eyrac":0,
"Barfleur":1,
"Boucau-Bayonne":2,
"Boulogne-sur-Mer":3,
"Brest":4,
"Calais":5,
"Cherbourg":6,
"Concarneau":7,
"Dielette":8,
"Dieppe":9,
"Dunkerque":10,
"Etretat":11,
"Fecamp":12,
"Granville":13,
"La Rochelle-Pallice":14,
"Le Havre":15,
"Le Touquet":16,
"Le Treport":17,
"Les Sables-d'Olonne":18,
"Ouistreham":19,
"Paimpol":20,
"Pointe de Grave":21,
"Portbail":22,
"Port-Navalo":23,
"Port-Tudy":24,
"Roscoff":25,
"Saint-Jean-de-Luz":26,
"Saint-Malo":27,
"Saint-Nazaire":28,
"Vieux-Boucau":29,
"Wissant":30,
          } 

   for col in df.select_dtypes('object').columns:
       df.loc[:,col] = df[col].map(code)       
   return df

In [59]:
#Drop des lignes NA's

def imputation(df):
    #df['is na'] = (df['Parainfluenza 3'].isna()) | (df['Leukocytes'].isna())
    #df = df.fillna(-999)
    df = df.dropna(axis=0)
    return  df


In [60]:
#Création de X, Y
def preprocessing(df):   
    df = imputation(df)
    df = encodage(df)  
    X = df.drop('mort', axis=1)
    y = df['mort']
    print(y.value_counts())
    return X, y

In [61]:
X, y = preprocessing(df) # on utilise 100% de nos données 

1    54097
0    54097
Name: mort, dtype: int64


### Modèles de prédictions en utilisant la 5 Fold Cross Validation

In [102]:
# random forest 
def RandomF(X,y):
    param_grid = {'n_estimators': np.arange(50,200,50), # le meilleur nbr d arbres est 1350, nbr d'arbres par defaut : 100 
                  'max_depth' : np.arange(1,10,1) , # la profondeur autorisée des arbres de décision individuels
                  'min_samples_split' : np.arange(1,10,1), # nombre minimal d’échantillons requis pour une division de nœud.
                  'criterion' : ['gini']} # critère, Gini est généralement la voie la plus rapide puisque l’entropie utilise un algorithme logarithmique.
    grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5) # créer la grille
    grid.fit(X,y) # entrainer cette grille
    print(grid.best_score_)  # voir le meilleur score 
    print(grid.best_params_) # voir les meilleurs paramètres
    modelRF = grid.best_estimator_ # sauvgarder ce model 
    return modelRF

# K plus proches voisins
def KNN(X,y):
    param_grid = {'n_neighbors': np.arange(1,30,1), # nombre de voisins
                  'metric': ['euclidean','manhattan','minkowski']} # mesure entre le voisin
                  #'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'], #algo utilisé pourcalculer les voisins les plus proches
                  #'weights':['uniform', 'distance'], #fonction de poids
                  #'leaf_size':np.arange(1,50,10)} # taille des feuilles # dictionnaire 
    grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5) # créer la grille
    grid.fit(X,y) # entrainer cette grille 
    print(grid.best_score_)  # voir le meilleur score 
    print(grid.best_params_) # voir les meilleurs paramètres
    modelKNN = grid.best_estimator_ # enregistrer le meilleur model 
    return modelKNN

# adaboost
def ADAB(X,y):    
    param_grid = {'n_estimators': np.arange(1,100,1), 
                  'learning_rate': np.arange(1,50,1) } # dictionnaire 
    # n_estimators : Nombre d’apprenants faibles à former de manière itérative
    # learning_rate : contribue au poids des apprenants faibles. Il utilise 1 comme valeur par défaut. 
    grid = GridSearchCV(AdaBoostClassifier(), param_grid, cv=5)
    grid.fit(X,y) # entrainer cette grille 
    print(grid.best_score_ ) # voir le meilleur score 
    print(grid.best_params_) # voir les meilleurs paramètres
    modelADABOOST = grid.best_estimator_ # enregistrer le meilleur model 
    return modelADABOOST

# Regression logistique
def RegL(X,y):
    param_grid = {'solver' : ['liblinear','linlinear','newton-cg','lbfgs','saga','sag'], 
                  'multi_class': ['ovr','multin'] ,
                  'class_weight' : [ 'balanced', 'None']} # dictionnaire 
    grid = GridSearchCV(LogisticRegression(), param_grid, cv=5) # créer la grille
    grid.fit(X,y) # entrainer cette grille 
    print(grid.best_score_)  # voir le meilleur score 
    grid.best_params_ # voir les meilleurs paramètres
    modelREGLOG = grid.best_estimator_ # enregistrer le meilleur model 
    return modelREGLOG

In [104]:
RandomF(X,y)

0.825073514599557
{'criterion': 'gini', 'max_depth': 9, 'min_samples_split': 6, 'n_estimators': 50}


RandomForestClassifier(max_depth=9, min_samples_split=6, n_estimators=50)